# Delegation Unit Test
Performing verification of the delegation mechanism by comparing the event data to the results of cadCAD model.

In [ ]:
import pandas as pd
import numpy as np

# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# print(module_path)
# sys.path.append(module_path)
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')


Configure environment

In [ ]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [ ]:
# print(delegation_events)

getcontext().prec = 6
print(getcontext())
delegation_events_dict = delegation_events
print("EXPECTED TRUTH FROM DELEGATION EVENTS:")
for timestep, events in delegation_events_dict.items():
    for event in events:        
        print(f"{timestep}, {event['delegator']}, {event['shares']}")
print(getcontext())



In [ ]:
timestep = 1
# df.iloc[timestep-1].delegators['0xd776a7306ee6a060cebb46b46d305e88fd39ba84'].shares
# df.iloc[timestep*6+1].delegators

# timestep += 1
# df.iloc[3].delegators['0xd776a7306ee6a060cebb46b46d305e88fd39ba84']
print("MODELED RESULTS")
print(getcontext())
delegation_event_shares = {}
for timestep, events in delegation_events_dict.items():
    for event in events:
        # curTimestepShares = df.iloc[timestep-1].delegators[event['delegator']].shares
        curTimestepShares = df.iloc[timestep-1].indexers[event['indexer']].delegators[event['delegator']].shares
        try:
            # lastTimestepShares = df.iloc[timestep-2].delegators[event['delegator']].shares
            lastTimestepShares = df.iloc[timestep-2].indexers[event['indexer']].delegators[event['delegator']].shares
        except:
            lastTimestepShares = 0
        deltaShares = curTimestepShares - lastTimestepShares
        delegation_event_shares[timestep] = deltaShares
        print(f"{timestep}, {event['delegator']}, {delegation_event_shares[timestep]}")



In [ ]:
print("UNITTEST RESULTS")
print(getcontext())
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, events in delegation_events_dict.items():
    for event in events:
        # curTimestepShares = df.iloc[timestep-1].delegators[event['delegator']].shares
        curTimestepShares = df.iloc[timestep-1].indexers[event['indexer']].delegators[event['delegator']].shares
        try:
            # lastTimestepShares = df.iloc[timestep-2].delegators[event['delegator']].shares
            lastTimestepShares = df.iloc[timestep-2].indexers[event['indexer']].delegators[event['delegator']].shares
        except:
            # this is a new delegator.
            lastTimestepShares = 0
        deltaShares = curTimestepShares - lastTimestepShares

        print(f"Event: Timestep={timestep}, Delegator ID={event['delegator']}, Shares Granted={event['shares']}")
        print(f"Model: Timestep={timestep}, Delegator ID={event['delegator']}, Shares Granted={deltaShares}")
        
        try:
            ratio = abs(event['shares'] / deltaShares)
        except:
            ratio = np.inf
        exact = ratio == 1.0
        close = 0.99 <= ratio <= 1.01
        reallyClose = 0.99999 <= ratio <= 1.00001

        if exact:            
            cntExact += 1
        elif reallyClose:
            cntReallyClose += 1
        elif close:
            cntClose += 1
        else:
            cntWrong += 1
        cnt += 1
        print("Shares Equal?", exact)
        print("Shares Within 0.001%?", reallyClose)
        print("Shares Within 1%?", close)
        print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
        print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   


# Conclusion
The number of shares granted by delegation in the first two events match the output of the cadCAD model exactly and then the next 3 are extremely close, diverging gradually.